# 주문 파일 저장

In [54]:
%%writefile ../src/Order.sol
pragma solidity ^0.6.0;

contract Order {
    uint unitPrice = 10;
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed quantity, string shipping);
    
    address payable owner;
    
    constructor() public {
        owner = msg.sender;
    }


    function order(bytes2 _itemId, uint quantity, string memory shipping) public payable isOwner {
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        
        emit OrderLog(msg.sender, _itemId, quantity, shipping);
    }
    
    
    function getBalance() view public returns(uint) {
        return address(this).balance;
    }
    
    modifier isOwner{
        require(msg.sender == owner);
        _;
    }
    
}

Overwriting ../src/Order.sol


In [1]:
!solc ../src/Order.sol --combined-json abi > ..\src\OrderABI.json

In [5]:
!type ..\src\OrderABI.json

{"contracts":{"../src/Order.sol:Order":{"abi":"[{\"inputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":true,\"internalType\":\"address\",\"name\":\"_from\",\"type\":\"address\"},{\"indexed\":false,\"internalType\":\"bytes2\",\"name\":\"_itemId\",\"type\":\"bytes2\"},{\"indexed\":true,\"internalType\":\"uint256\",\"name\":\"quantity\",\"type\":\"uint256\"},{\"indexed\":false,\"internalType\":\"string\",\"name\":\"shipping\",\"type\":\"string\"}],\"name\":\"OrderLog\",\"type\":\"event\"},{\"inputs\":[],\"name\":\"getBalance\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"bytes2\",\"name\":\"_itemId\",\"type\":\"bytes2\"},{\"internalType\":\"uint256\",\"name\":\"quantity\",\"type\":\"uint256\"},{\"internalType\":\"string\",\"name\":\"shipping\",\"type\":\"string\"}],\"name\":\"order\",\"outputs\":[],\"stat

In [2]:
!solc ../src/Order.sol --combined-json bin > ..\src\OrderBIN.json

## 배포

In [3]:
%%writefile ../src/OrderDeploy.js
var Web3=require('web3');
var _abiJson = require('./OrderABI.json');
var _binJson = require('./OrderBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);
_bin=_binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();

    //gas check
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/OrderDeploy.js


In [4]:
!node ../src/OrderDeploy.js

- contract name:  ../src/Order.sol:Order
>> gas: 259536
hash: 0x58039558c324814b1ae2ddf9db2b6021c68b3d2f08aaaac89f0bfc19315dbf75
---> The contract deployed to: 0xdC1F58AD69F815D76243332DD4cDB00931C899f6


## 사용

In [33]:
%%writefile ../src/OrderUse.js
var Web3=require('web3');
var fs=require('fs');
var _abiJson = require('./OrderABI.json');
var _binJson = require('./OrderBIN.json');


var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _order = new web3.eth.Contract(_abiArray, '0xA1Bd56EB57beDa3211E8fd41263a90F1020A7c8F');

var event = _order.events.OrderLog({filter: {_from: web3.eth.accounts[0], _value: 30}, fromBlock: 0}, function (error, result) {
    if (!error) {
        var addr = JSON.stringify(result.returnValues[0])
        var product = JSON.stringify(result.returnValues[1])
        var amount = JSON.stringify(result.returnValues[2])
        var shipping = JSON.stringify(result.returnValues[3])
        var log = `address: ${addr} \nproduct: ${product} \namount: ${amount} \nshipping: ${shipping}`
        
        console.log("Event fired: " + log);
        console.log()
        
        fs.writeFile("../src/OrderLog.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file\n");
            }
        });
        
        //대기 후 종료
        setTimeout(function() {
          process.exit(1);
        }, 1000);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();

    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    
    // this will fire an event
    my = await _order.methods.order("0x1234", 3, "20 2-gil Hongji-dong Jongro-gu Busan")
        .send({from: accounts[0], gas: 100000, value:30})
    console.log("---> MyFunction called ");

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting ../src/OrderUse.js


In [34]:
!node ../src/OrderUse.js

- contract name:  ../src/Order.sol:Order
Event fired: address: "0xa46e26d9BD272D8784047055271C243533552eCB" 
product: "0x1234" 
amount: "3" 
shipping: "20 2-gil Hongji-dong Jongro-gu Busan"

>> Writing to file

Event fired: address: "0xa46e26d9BD272D8784047055271C243533552eCB" 
product: "0x1234" 
amount: "3" 
shipping: "20 2-gil Hongji-dong Jongro-gu Busan"

>> Writing to file

---> MyFunction called 
Balance after: 99982704343999993730
Balance diff: 559140000006144


In [35]:
!type ..\src\OrderLog.txt

address: "0xa46e26d9BD272D8784047055271C243533552eCB" 
product: "0x1234" 
amount: "3" 
shipping: "20 2-gil Hongji-dong Jongro-gu Busan"


# geth 배포 및 사용

In [85]:
!type ..\src\OrderGethBIN.json

{"contracts":{"../src/Order.sol:Order":{"bin":"6080604052600a60005534801561001557600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506102f9806100666000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a92914610059575b600080fd5b34801561003a57600080fd5b50610043610149565b6040518082815260200191505060405180910390f35b6101476004803603606081101561006f57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100c157600080fd5b8201836020820111156100d357600080fd5b803590602001918460018302840111640100000000831117156100f557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610151565b005b600047905090565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673f

In [27]:
%%writefile ..\src\OrderGethBIN.json
{"contracts":{"../src/Order.sol:Order":{"bin":"6080604052600a60005534801561001557600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506102f9806100666000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a92914610059575b600080fd5b34801561003a57600080fd5b50610043610149565b6040518082815260200191505060405180910390f35b6101476004803603606081101561006f57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100c157600080fd5b8201836020820111156100d357600080fd5b803590602001918460018302840111640100000000831117156100f557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610151565b005b600047905090565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146101ab57600080fd5b60008054830290508034146101bf57600080fd5b823373ffffffffffffffffffffffffffffffffffffffff167f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a868560405180837dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200180602001828103825283818151815260200191508051906020019080838360005b83811015610282578082015181840152602081019050610267565b50505050905090810190601f1680156102af5780820380516001836020036101000a031916815260200191505b50935050505060405180910390a35050505056fea26469706673582212200118e129125705772c70cde480a91185588696347bc65514cf812f41a93d10b064736f6c63430006030033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Overwriting ..\src\OrderGethBIN.json


### geth 배포

In [28]:
%%writefile ../src/OrderGethDeploy.js
var Web3=require('web3');
var _abiJson = require('./OrderABI.json');
var _binJson = require('./OrderGethBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));


contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);
_bin=_binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/OrderGethDeploy.js


```
geth --exec "personal.unlockAccount(eth.accounts[0], '1111')" attach http://localhost:8445
miner.start(1);admin.sleepBlocks(1);miner.stop()
```

In [29]:
!node ../src/OrderGethDeploy.js

- contract name:  ../src/Order.sol:Order
Deploying the contract from 0x5C46B33Dc12cF6ebbb834310EF235730f6C8449e
hash: 0x3d70901d4fc233fa1fe4450994641a5aa4edfc4e3ab98ef47090ef45eb41db2f
---> The contract deployed to: 0xAe4cbe46c5f8F04463D9B1C266FB8a60249d8Bf0


### geth 사용

In [38]:
%%writefile ../src/OrderGethUse.js
var Web3=require('web3');
var fs=require('fs')
var _abiJson = require('./OrderABI.json');

var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8546"));
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _order = new web3.eth.Contract(_abiArray, '0xe5673EA8992cb4ea3b25ff7448eD3861abA4C184');

var event = _order.events.OrderLog({filter: {_from: web3.eth.accounts[0], _value: 30}, fromBlock: 0}, function (error, result) {
    if (!error) {
        var log = JSON.stringify(result.returnValues)
        console.log("Event fired: " + log);
        console.log()
        
        fs.writeFile("../src/OrderLog.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file\n");
            }
        });
        
        //대기 후 종료
        setTimeout(function() {
          process.exit(1);
        }, 1000);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();

    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    
    // this will fire an event
    my = await _order.methods.order("0x1234", 3, "20 2-gil Hongji-dong Jongro-gu Busan")
        .send({from: accounts[0], gas: 100000, value:30})
    console.log("---> MyFunction called ");

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
    
    //대기 후 종료
    setTimeout(function() {
      process.exit(1);
    }, 1000);
}

doIt()

Overwriting ../src/OrderGethUse.js


In [42]:
!node ../src/OrderGethUse.js

- contract name:  ../src/Order.sol:Order
---> MyFunction called 
Balance after: 9999999999999999970
Balance diff: -5000000000000000000
